## Calculate PI

In this exercise you will get to practice using both broadcast and reduce MPI routines. The program will calculate the value of pi derived from integral by approximation using Riemann sum. 

The program takes the following steps:

1. Root process (process 0) asks the user for the number of integral intervals (for interactive notebooks we have hardcoded this number) and then broadcasts this number to all of the other processes.

2. Each process then locally adds up every n’th interval.

3. Finally, the sums computed are added together using reduction.

***
#### C skeleton

In [ ]:
%%file pi.c
#include <stdio.h>
#include <mpi.h>

int main()
{
    int rank, size, i;
    double PI_value = 3.141592653589793238462643;
    double mypi, pi, w, sum, x;

    MPI_Init(NULL, NULL);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    int n = 0;
    if (rank == 0) {
        //printf("Enter the number of intervals:");
        //scanf("%d",&n);
        if (n==0) n = 1024*size;
    }
    // TODO: broadcast

    w = 1.0 / n;
    sum = 0.0;
    for (i = rank + 1; i <= n; i += size) {
        x = w * ((double)i - 0.5);
        sum += 4.0 / (1.0 + x*x);
    }
    mypi = w * sum;

    // TODO: reduce

    if (rank == 0) {
        printf("pi is approximately %.16f, Error is %.16f\n",
           pi, fabs(pi - PI_value));
    }

    MPI_Finalize();
}

Now compile it and run it with 4 processes. 

In [ ]:
!mpicc pi.c -o pi && mpirun -np 4 --allow-run-as-root pi

***
#### Python skeleton

In [ ]:
%%file pi.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
PI_value = 3.141592653589793238462643

n = 0
if rank == 0:
    n = 1024*size
# TODO: broadcast

w = 1/n
sum = 0;
for i in range (rank+1, n, size):
    x = w * (i - 0.5)
    sum += 4.0 / (1.0 + x*x)
mypi = w * sum

# TODO: reduce

if rank == 0:
    print("pi is approximately %.16f, Error is %.16f" % (pi,
        abs(pi - PI_value)))

Now compile it and run it with 4 processes. 

In [ ]:
!mpirun -np 4 --allow-run-as-root python pi.py

***
#### Fortran skeleton

In [ ]:
%%file pi.f90
program pi
use mpi

integer ( kind = 4 ) error
integer :: i, rank, size, n
double precision :: PI_value, mypi, rootpi, w, sum, x
PI_value = 3.141592653589793238462643

call MPI_Init(error)
call MPI_Comm_rank(MPI_COMM_WORLD, rank, error)
call MPI_Comm_size(MPI_COMM_WORLD, size, error)

n = 0
if (rank == 0) then
    n = 1024*size
endif
! TODO: broadcast

w = 1.0 / n
sum = 0.0
do i = rank+1, n, size
    x = w * (i-0.5)
    sum = sum + 4.0 / (1.0 + x*x)
end do
mypi = w * sum

! TODO: reduce

if (rank == 0) then
    print *, "pi is approximately", rootpi, "Error is", abs(rootpi - PI_value)
endif

call MPI_Finalize(error)
end

Now compile it and run it with 4 processes. 

In [ ]:
!mpif90 pi.f90 && mpirun -np 4 --allow-run-as-root a.out

***

### You can compare with our solution:

***
#### C solution

In [ ]:
%%file pi.c
#include <stdio.h>
#include <mpi.h>

int main()
{
    int rank, size, i;
    double PI_value = 3.141592653589793238462643;
    double mypi, pi, w, sum, x;

    MPI_Init(NULL, NULL);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    int n = 0;
    if (rank == 0) {
        //printf("Enter the number of intervals:");
        //scanf("%d",&n);
        if (n==0) n = 1024*size;
    }
    MPI_Bcast(&n, 1, MPI_INT, 0, MPI_COMM_WORLD);

    w = 1.0 / n;
    sum = 0.0;
    for (i = rank + 1; i <= n; i += size) {
        x = w * ((double)i - 0.5);
        sum += 4.0 / (1.0 + x*x);
    }
    mypi = w * sum;

    MPI_Reduce(&mypi, &pi, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);

    if (rank == 0) {
        printf("pi is approximately %.16f, Error is %.16f\n",
           pi, fabs(pi - PI_value));
    }

    MPI_Finalize();
}

In [ ]:
!mpicc pi.c -o pi && mpirun -np 4 --allow-run-as-root pi

***
#### Python solution

In [ ]:
%%file pi.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
PI_value = 3.141592653589793238462643

n = 0
if rank == 0:
    n = 1024*size
n = comm.bcast(n, root=0)

w = 1/n
sum = 0;
for i in range (rank+1, n, size):
    x = w * (i - 0.5)
    sum += 4.0 / (1.0 + x*x)
mypi = w * sum

pi = comm.reduce(mypi, op=MPI.SUM, root=0)

if rank == 0:
    print("pi is approximately %.16f, Error is %.16f" % (pi,
        abs(pi - PI_value)))

In [ ]:
!mpirun -np 4 --allow-run-as-root python pi.py

***
#### Fortran solution

In [ ]:
%%file pi.f90
program pi
use mpi

integer ( kind = 4 ) error
integer :: i, rank, size, n
double precision :: PI_value, mypi, rootpi, w, sum, x
PI_value = 3.141592653589793238462643

call MPI_Init(error)
call MPI_Comm_rank(MPI_COMM_WORLD, rank, error)
call MPI_Comm_size(MPI_COMM_WORLD, size, error)

n = 0
if (rank == 0) then
    n = 1024*size
endif
call MPI_Bcast(n, 1, MPI_INT, 0, MPI_COMM_WORLD, error)

w = 1.0 / n
sum = 0.0
do i = rank+1, n, size
    x = w * (i-0.5)
    sum = sum + 4.0 / (1.0 + x*x)
end do
mypi = w * sum

call MPI_Reduce(mypi, rootpi, 1, MPI_DOUBLE_PRECISION, MPI_SUM, 0, MPI_COMM_WORLD, error)    

if (rank == 0) then
    print *, "pi is approximately", rootpi, "Error is", abs(rootpi - PI_value)
endif

call MPI_Finalize(error)
end

In [ ]:
!mpif90 pi.f90 && mpirun -np 4 --allow-run-as-root a.out